In [1]:
from joblib.numpy_pickle_utils import xrange
from sklearn.datasets import fetch_openml
import numpy as np
mnist = fetch_openml('mnist_784')
x,y=mnist['data'],mnist['target']

In [549]:
model=Digit_Classifier(x,y,'sigmoid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


0.9773
0.9841
0.9543
0.9353
0.9504
0.95
0.9736
0.9725
0.9071
0.9295


In [555]:
model1=Digit_Classifier(x,y,'tanh')

0.9679
0.9821
0.9321
0.9375
0.9508
0.92
0.9584
0.9688
0.9026
0.8991


In [567]:
model1.predictor(x[60005])

1


In [568]:
y[60005]

'1'

In [548]:
class Digit_Classifier:
  def __init__(self,x,y,f):
    self.digits_locator=[]
    for i in range(10):
      self.digits_locator.append(Digit_Neuron())
    self.active=f
    for i in range(10):
      x_training,y_training,x_test,y_test=data_maker(x,y,self.active,i)
      self.digits_locator[i].learn_data(x_training,y_training,0.001,self.active,x_test,y_test,str(i))
    
  def predictor(self,x):
    for i in range(10):
      result=self.digits_locator[i].predict(x,self.active)
      if result==1:
        print(i)

  def data_maker(x,y,activation,target_label):
    option=np.zeros(2)
    if activation=='sigmoid':
      option[0]=0
      option[1]=1
    elif activation=='tanh':
      option[0]=-1
      option[1]=1
    x_training=x[:60000]
    y_training=y[:60000]
    x_test=x[60000:]
    y_test=y[60000:]
    y_training = np.array(y_training).T
    y_training=y_training.astype(str).astype(int)
    for i in range(y_training.size):
      if y_training[i]==target_label:
         y_training[i]=option[1]
      else:
        y_training[i]=option[0]
    return x_training,y_training,x_test,y_test
  

In [501]:
for i in range(10):
  print(i)

0
1
2
3
4
5
6
7
8
9


In [502]:
class Digit_Neuron:
  def __init__(self):
    self.arr=np.zeros(785)

  def sigmoid(value):
    z = 1/(1 + np.exp(-value))
    return z

  def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x)) 
    
  def classify(arr,weight,activation):
    prd=np.dot(arr,weight[1:])+weight[0]
    if activation=='sigmoid':
      return sigmoid(prd)
    elif activation=='tanh':
      return tanh(prd)

  def tanh(value):
    return np.tanh(value)

  def tanh_derivative(value):
    return 1-(tanh(value)*tanh(value))

  def predict(self,data,activation):
    result=np.dot(data,self.arr[1:])+self.arr[0]
    if activation=='sigmoid':
      return (sigmoid(result))
    elif activation=='tanh':
      if result>=-9999999999:
        return 1
      else:
        return 0
  def accuracy_checker(arr,x_test,y_test,active,target_label):
    counter=0
    for i in range(y_test.size):
      if predict(x_test[i],arr,active)==1 and y_test[i]==target_label:
        counter+=1
      elif predict(x_test[i],arr,active)==0 and y_test[i]!=target_label:
        counter+=1
    return (counter/y_test.size)
  
  def learn_data(self,x,t,learning_rate,activation,x_test,y_test,target):
    result=np.zeros(self.arr.size)
    counter=10
    max=-1
    while counter>0:
      output=classify(x,self.arr,activation)
      error=t-output
      if activation=='sigmoid':
        term=np.dot(x.T,error*sigmoid_derivative(output))
        self.arr[1:]+=term
        self.arr[0]+=(error*sigmoid_derivative(output)).sum()
        val=accuracy_checker(self.arr,x_test,y_test,'sigmoid',target)
        if max<val:
          max=val
          result[1:]=self.arr[1:]
          result[0]=self.arr[0]
      else:
        term=np.dot(x.T,error*tanh_derivative(output))
        self.arr[1:]+=term
        self.arr[0]+=(error*tanh_derivative(output)).sum()
        val=accuracy_checker(self.arr,x_test,y_test,'tanh',target)
        if max<val:
          max=val
          result[1:]=self.arr[1:]
          result[0]=self.arr[0]
      counter=counter-1
    self.arr[1:]=result[1:]
    self.arr[0]=result[0]
    print(accuracy_checker(self.arr,x_test,y_test,activation,target))

